# Week 3 - Augmented Judgment

intro

In [3]:
#All these packages need to be installed from pip
#For ML
import sklearn
import sklearn.feature_extraction.text
import sklearn.decomposition
from sklearn import preprocessing, linear_model
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.datasets import fetch_20newsgroups, make_blobs
from sklearn.feature_extraction.text import TfidfVectorizer  #Feature extraction
from sklearn.naive_bayes import MultinomialNB #Our learner.
from sklearn.pipeline import make_pipeline
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestRegressor
from sklearn.neural_network import MLPClassifier
from sklearn import neighbors

#Displays the graphs
import graphviz #You also need to install the command line graphviz

#For NLP
import nltk
from nltk.tag import StanfordNERTagger
from nltk.tag import StanfordPOSTagger
from nltk.parse import stanford
from nltk.tokenize import word_tokenize
from nltk.tree import Tree
from nltk.draw.tree import TreeView
from nltk.tokenize import sent_tokenize


import numpy as np #arrays
import matplotlib.pyplot as plt #Plots
from matplotlib.colors import ListedColormap
import seaborn #Makes plots look nice, also heatmaps
import scipy as sp #for interp

%matplotlib inline

#These are from the standard library
import collections
import os
import os.path
import random
import re
import glob
import pandas
import requests
import json
import math
import tarfile
import zipfile
import io

We also develop a couple of functions for splitting our text data:

In [ ]:
def split_data(data, prob):
    """split data into fractions [prob, 1 - prob]"""
    results = [], []
    for row in data:
        results[0 if random.random() < prob else 1].append(row)
    return results

# The Naïve Bayes Classifier

## Naive Bayes from Scratch

First, let's build a Naive Bayes classifier from scratch to illustrate what's in
a classifier. This example is drawn from *Data Science from Scratch* by Joel
Grus.

### Mathematical Preliminaries

Naïve Bayes contains an independence assumption that the probability of features
conditional on classification $S$ is equivalent to the product of conditional
probabilities associated with each of those features, hence this model's
*naïvité*: $P(X_1 = x_1,\dots,X_n = x_n\,|\,S) = P(X_1 = x_1\,|\,S)\times \dots
    \times P(X_n = x_n\,|\,S)$

To be concrete, let's assume we are building a spam filter.

Given a vocabulary $w_1,\dots,w_n$, let $X_i$ be the event "message contains"
$w_i$." $X_i = x_i, x_i \in \{0,1\}$.

$S$ is the event "message is spam" and $\neg S$ is the event "message is not
spam."

According to Bayes' Theorem:

$P(S\,|\,X_1 = x_1,\dots, X_n = x_n) = \frac{P(X_1 = x_1,\dots, X_n =
x_n\,|\,S)P(S)}{P(X_1 = x_1,\dots, X_n = x_n)} = \frac{P(X_1 = x_1,\dots, X_n =
x_n\,|\,S)P(S)}{P(X_1 = x_1,\dots, X_n = x_n\,|\,S)P(S)\, + \,P(X_1 = x_1,\dots,
X_n = x_n\,|\,\neg S)P(\neg S)}$

We further assume that we have no knowledge of the prior probability of spam; so
$P(S) = P(\neg S) = 0.5$ (this is the principle of indifference)

With this simplification, $P(S\,|\,X_1 = x_1,\dots, X_n = x_n) = \frac{P(X_1 =
x_1,\dots, X_n = x_n\,|\,S)}{P(X_1 = x_1,\dots, X_n = x_n\,|\,S)\, +\, P(X_1 =
x_1,\dots, X_n = x_n\,|\,\neg S)}$

Now we make the Naive Bayes assumption: $P(X_1 = x_1,\dots,X_n = x_n\,|\,S) =
P(X_1 = x_1\,|\,S)\times \dots
    \times P(X_n = x_n\,|\,S)$

We can estimate $P(X_i = x_i\,|\,S)$ by computing the fraction of spam messages
containing the word $i$, e.g., Obamacare.

Smoothing: $P(X_i\,|\,S) = \frac{(k + \textrm{number of spams containing}\,
w_i)}{(2k + \textrm{number of spams})}$



### Now, to code this up...

Lets start with spam detection, we have data from:
https://spamassassin.apache.org/publiccorpus/ in `data/Spam_Data` with the spam
in `data/Spam_Data/20021010_spam.tar.bz2` and the non spam in `data/Spam_Data/20021010_easy_ham.tar.bz2` and `data/Spam_Data/20021010_easy_ham.tar.bz2` zip files.

In [ ]:
def loadEmailZip(targetFile, category):
    # regex for stripping out the leading "Subject:" and any spaces after it
    subject_regex = re.compile(r"^Subject:\s+")

    #The dict that will become the DataFrame
    emailDict = {
        'category' : [],
        'text' : [],
    }
    with tarfile.open(targetFile) as tar:
        for tarinfo in tar.getmembers():
            if tarinfo.isreg():
                with tar.extractfile(tarinfo) as f:
                    s = f.read().decode('latin1', 'surrogateescape')
                    for line in s.split('\n'):
                        if line.startswith("Subject:"):
                            #Could also save the subject field
                            subject = subject_regex.sub("", line).strip()
                            emailDict['text'].append(subject)
    emailDict['category'] = [category] * len(emailDict['text'])
    return pandas.DataFrame(emailDict)  

spamDF = loadEmailZip('data/Spam_Data/20021010_spam.tar.bz2', 'spam')
spamDF = spamDF.append(loadEmailZip('data/Spam_Data/20021010_hard_ham.tar.bz2', 'not spam'), ignore_index= True)
spamDF = spamDF.append(loadEmailZip('data/Spam_Data/20021010_easy_ham.tar.bz2', 'not spam'), ignore_index= True)
spamDF['is_spam'] = [c == 'spam' for c in spamDF['category']]
spamDF[:10]

We can now tokenize and normalize like normal

In [ ]:
def normlizeTokens(tokenLst, stopwordLst = None, stemmer = None, lemmer = None, vocab = None):
    #We can use a generator here as we just need to iterate over it

    #Lowering the case and removing non-words
    workingIter = (w.lower() for w in tokenLst if w.isalpha())

    #Now we can use the semmer, if provided
    if stemmer is not None:
        workingIter = (stemmer.stem(w) for w in workingIter)

    #And the lemmer
    if lemmer is not None:
        workingIter = (lemmer.lemmatize(w) for w in workingIter)

    #And remove the stopwords
    if stopwordLst is not None:
        workingIter = (w for w in workingIter if w not in stopwordLst)

    #We will return a list with the stopwords removed
    if vocab is not None:
        vocab_str = '|'.join(vocab)
        workingIter = (w for w in workingIter if re.match(vocab_str, w))

    return list(workingIter)

stop_words_nltk = nltk.corpus.stopwords.words('english')
snowball = nltk.stem.snowball.SnowballStemmer('english')

spamDF['tokenized_emails'] = spamDF['text'].apply(lambda x: nltk.word_tokenize(x))
spamDF['normalized_emails'] = spamDF['tokenized_emails'].apply(lambda x: normlizeTokens(x, stopwordLst = None, stemmer = None))
spamDF[:5]

Now we need to count the number of times each word shows up in spam and non-spam

In [ ]:
def count_words(traingDF, textColumn, trainingColumn):
    counts = collections.defaultdict(lambda: [0, 0])
    for index, row in traingDF.iterrows():
        for word in set(row[textColumn]):
            if row[trainingColumn]:
                counts[word][0] += 1
            else:
                counts[word][1] += 1
    return counts
wordCounts = count_words(spamDF, 'normalized_emails', 'is_spam')
wordCounts['cancel']

We need a function to convert these counts into (smoothed) probabilities

In [ ]:
def word_probabilities(counts, total_spams, total_non_spams, k=0.5):
    """turn the word_counts into a list of triplets
    w, p(w | spam) and p(w | ~spam)"""
    retTuples = []
    for w, (spam, non_spam) in counts.items():
        retTuples.append((w,
             (spam + k) / (total_spams + 2 * k),
             (non_spam + k) / (total_non_spams + 2 * k)))
    return retTuples

Now we need to come up with a way to compute the spam probability for a message,
given word probabilities. With the Naive Bayes assumption, we *would* be
multiplying together a bunch of probabilities. This is bad (e.g., floating point
underflow) so we instead compute:

$p_1 *\dots*p_n = \exp(\, \log(p_1) + \dots + \log(p_n)\,)$; recall $\log(ab) =
\log a + \log b$ and $\exp(\, \log x \,) = x$

Thank you, John Napier (1550-1617), for inventing logs.

In [ ]:
def spam_probability(word_probs, message_words):
    #message_words = tokenize(message)
    log_prob_if_spam = log_prob_if_not_spam = 0.0 #Initialize; we are working with log probs to deal with underflow.

    for word, prob_if_spam, prob_if_not_spam in word_probs: #We iterate over all possible words we've observed
        # for each word in the message,
        # add the log probability of seeing it
        if word in message_words:
            log_prob_if_spam += math.log(prob_if_spam) #This is prob of seeing word if spam
            log_prob_if_not_spam += math.log(prob_if_not_spam) #This is prob of seeing word if not spam

        # for each word that's not in the message
        # add the log probability of _not_ seeing it
        else:
            log_prob_if_spam += math.log(1.0 - prob_if_spam)
            log_prob_if_not_spam += math.log(1.0 - prob_if_not_spam)
    P = 1/(1 + math.exp(log_prob_if_not_spam - log_prob_if_spam))
    #prob_if_spam = math.exp(log_prob_if_spam) #Compute numerator
    #prob_if_not_spam = math.exp(log_prob_if_not_spam)
    #return prob_if_spam / (prob_if_spam + prob_if_not_spam) #Compute whole thing and return
    return P

Think: how would this change if $P(S) \neq P(\neg S)$

Now we write a class (this is a Python term) for our Naive Bayes Classifier

In [ ]:
class NaiveBayesClassifier:

    def __init__(self, k=0.5):
        self.k = k
        self.word_probs = [] #Initializes word_probs as an empty list, sets a default smoothing parameters

    def train(self, training_set, trainingColumn, textColumn): #Operates on the training_set

        # count spam and non-spam messages: first step of training
        num_spams = training_set[trainingColumn].value_counts()[True]
        num_non_spams = len(training_set) - num_spams

        # run training data through our "pipeline"
        word_counts = count_words(training_set, textColumn, trainingColumn)
        self.word_probs = word_probabilities(word_counts,
                                             num_spams,
                                             num_non_spams,
                                             self.k) #"Train" classifier

    def classify(self, message):
        return spam_probability(self.word_probs, message) #Now we have all we need to classify a message


To train and then evaluate our model, we will split our data into ***training***
& ***testing*** sets.

In [ ]:
np.random.seed(0)
shuffledSpamDF = spamDF.reindex(np.random.permutation(spamDF.index))
holdBackFraction = .2
holdBackIndex = int(holdBackFraction * len(shuffledSpamDF))
train_data = shuffledSpamDF[holdBackIndex:].copy()
test_data = shuffledSpamDF[:holdBackIndex].copy()

In [ ]:
classifier = NaiveBayesClassifier()

In [ ]:
classifier.train(train_data, 'is_spam', 'normalized_emails')

In [ ]:
len(train_data)

In [ ]:
train_data[:5]

Some simple evaluation:

In [ ]:
# triplets (subject, actual is_spam, predicted spam probability)
classified = [(row['category'], classifier.classify(row['normalized_emails']))
              for index, row in test_data.iterrows()]

# assume that spam_probability > 0.5 corresponds to spam prediction # and count the combinations of (actual is_spam, predicted is_spam)
counts = collections.Counter((actual == 'spam', predicted_probability > 0.5)
                     for actual, predicted_probability in classified)

Now, let's see how we did! We will do so by computing the Precision, Recall and
F-measure or harmonic mean between Precision and Recall.

In [ ]:
counts

Precision:

In [ ]:
precision = counts[(True,True)]/(counts[(False,True)]+counts[(True,True)]) #True positives over all positive predictions
print(precision)

Recall:

In [ ]:
recall = counts[(True,True)]/(counts[(True,False)]+counts[(True,True)])#what fraction of positives identified
print(recall)

F-measure:

In [ ]:
f_measure = 2 * (precision * recall)/(precision + recall)
print (f_measure)

Let's look at how the emails are classified.

In [ ]:
test_data['predicted_spam_probability'] = [classifier.classify(d) for d in test_data['normalized_emails']]

In [ ]:
test_data[:5]

Now, we calculate the ROC curve, and integrate to get the area under the curve
(AUC), or probability mass associated with correct classification.

In [ ]:
x, y, _ = sklearn.metrics.roc_curve(test_data['is_spam'], test_data['predicted_spam_probability'])
roc_auc = sklearn.metrics.auc(x,y)

In [ ]:
plt.figure()
plt.plot(x,y, color = 'darkorange', lw = 2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic curve')
plt.legend(loc="lower right")
plt.show()

What will happen if we split the data differently (e.g.,
less training data, more testing data).

<span style="color:red">**Try it!**

We can also find words that lead to a high probability of spam (using Bayes'
Theorem):

In [ ]:
def p_spam_given_word(word_prob):
    """uses bayes's theorem to compute p(spam | message contains word)"""
    # word_prob is one of the triplets produced by word_probabilities

    word, prob_if_spam, prob_if_not_spam = word_prob
    return prob_if_spam / (prob_if_spam + prob_if_not_spam)

In [ ]:
words = sorted(classifier.word_probs,key=p_spam_given_word)

In [ ]:
spammiest_words = words[-15:]
hammiest_words = words[:15]

In [ ]:
spammiest_words

In [ ]:
hammiest_words

# Now with a corpus of Clinton / Obama Press Releases

While the Clinton and Obama Senatorial Press are not hand-coded, we can imagine
that we have been given a stack of such press releases, but lost the metadata
associated with which senatorial office issued which. If we label a few of them,
how well can our classifier do at recovering the rest?

In [ ]:
ObamaClintonReleases = pandas.read_csv("data/ObamaClintonReleases.csv")
ObamaClintonReleases = ObamaClintonReleases.dropna(axis=0, how='any')
ObamaClintonReleases['tokenized_text'] = ObamaClintonReleases['text'].apply(lambda x: nltk.word_tokenize(x))
ObamaClintonReleases['normalized_text'] = ObamaClintonReleases['tokenized_text'].apply(lambda x: normlizeTokens(x, stopwordLst = stop_words_nltk, stemmer = snowball))

Let's turn the 'targetSenator' column into a binary variable.

In [ ]:
ObamaClintonReleases['IsObama'] = [s == 'Obama' for s in ObamaClintonReleases['targetSenator']]

Let's split the data into training data and test data.

In [ ]:
np.random.seed(0)
ObamaClintonReleases = ObamaClintonReleases.reindex(np.random.permutation(ObamaClintonReleases.index))
holdBackFraction = .25
holdBackIndex = int(holdBackFraction * len(ObamaClintonReleases))
train_data = ObamaClintonReleases[holdBackIndex:].copy()
test_data = ObamaClintonReleases[:holdBackIndex].copy()

In [ ]:
print (len(train_data))
print (len(test_data))

## Logistic Regression

First, let's try with a logistic regression, which may be familiar to you from
statistical methods classes. First, we must turn the training dataset into a tf-
idf matrix:

In [ ]:
TFVectorizer = sklearn.feature_extraction.text.TfidfVectorizer(max_df=100, min_df=2, stop_words='english', norm='l2')
TFVects = TFVectorizer.fit_transform(train_data['text'])

Note that we can use the CountVectorizer instead, which simply produces a matrix
of word counts.

In [ ]:
TFVects.shape

In a regression, we cannot have more variables than cases. So, we need to first
do a dimension reduction. First, we will approah this with PCA. You have
previously seen this in week 3. Here we are not concerned about visualization
but rather classification and so all principal components are calculated.

In [ ]:
PCA = sklearn.decomposition.PCA
pca = PCA().fit(TFVects.toarray())
reduced_data = pca.transform(TFVects.toarray())

Visualization in 2D:

In [ ]:
colordict = {
True: 'red',
False: 'blue',
    }
colors = [colordict[c] for c in train_data['IsObama']]
fig = plt.figure(figsize = (5,3))
ax = fig.add_subplot(111)
ax.set_frame_on(False)
ax.scatter(reduced_data[:, 0], reduced_data[:, 1], color = colors, alpha = 0.5, label = colors)
plt.xticks(())
plt.yticks(())
plt.title('True Classes, Training Set')
plt.show()

PCA cannot distinguish Clinton from Obama very well. Let's perform a screeplot
to see how many dimensions we need.

In [ ]:
n = TFVects.shape[0]
fig = plt.figure(figsize=(12,5))
ax1 = fig.add_subplot(121)
eigen_vals = np.arange(n) + 1
ax1.plot(eigen_vals, pca.explained_variance_ratio_, 'ro-', linewidth=2)
ax1.set_title('Scree Plot')
ax1.set_xlabel('Principal Component')
ax1.set_ylabel('Proportion of Explained Variance')

ax2 = fig.add_subplot(122)
eigen_vals = np.arange(20) + 1
ax2.plot(eigen_vals, pca.explained_variance_ratio_[:20], 'ro-', linewidth=2)
ax2.set_title('Scree Plot (First 20 Principal Components)')
ax2.set_xlabel('Principal Component')
ax2.set_ylabel('Proportion of Explained Variance')
plt.show()

Let's choose the first 10 pricipal components as our covariates.

In [ ]:
X = reduced_data[:, :10]

Transform our predictor variable.

In [ ]:
Y = np.array([int(label) for label in train_data['IsObama']])

Now we fit a logistic regression to our data.

In [ ]:
logistic = linear_model.LogisticRegression()
logistic.fit(X, Y)

Let's see how the logistic regression performs on the training dataset on we
develop the model. Unfortunately, mean accuracy is only about 68%.

In [ ]:
logistic.score(X,Y)

How does it perform on the testing dataset, which we "held out" and did not use
for model training?

In [ ]:
TFVects_test = TFVectorizer.transform(test_data['text'])
reduced_data_test = pca.transform(TFVects_test.toarray())
X_test = reduced_data_test[:, :10]
Y_test = np.array([int(label) for label in test_data['IsObama']])
logistic.score(X_test, Y_test)

Slightly poorer. How about using more dimensions (40)?

In [ ]:
X = reduced_data[:, :40]
logistic.fit(X, Y)
X_test = reduced_data_test[:, :40]
print(logistic.score(X,Y))
print(logistic.score(X_test, Y_test))

Or still more (100)?

In [ ]:
X = reduced_data[:, :100]
logistic.fit(X, Y)
X_test = reduced_data_test[:, :100]
print(logistic.score(X,Y))
print(logistic.score(X_test, Y_test))

Or even more (200)!

In [ ]:
X = reduced_data[:, :200]
logistic.fit(X, Y)
X_test = reduced_data_test[:, :200]
print(logistic.score(X,Y))
print(logistic.score(X_test, Y_test))

This is becoming ridiculous (400)!

In [ ]:
X = reduced_data[:, :400]
logistic.fit(X, Y)
X_test = reduced_data_test[:, :400]
print(logistic.score(X,Y))
print(logistic.score(X_test, Y_test))

Increasing the number of covariates would overfit our data, and it seems that
using a logistic regression, our prediction accuracy is at best about 80%. We
can, however, try a logistic regression that uses L1 regularization or the
L1-norm loss function, which is also known as least absolute deviations (LAD),
least absolute errors (LAE) or L1 penalty. It minimizes the sum of the absolute
differences (S) between the target value ($Y_i$) and the estimated values
($f(x_i)$):

$S=\sum^n_{i=1}|y_i=f(x_i)|$

The result is a model retaining only the most individually significant features.

In [ ]:
X = TFVects
logistic_l1= linear_model.LogisticRegression(penalty='l1')
logistic_l1.fit(X, Y)
print(logistic_l1.score(X,Y))

Train the model using training data, and then test it on the testing data.

In [ ]:
print(logistic_l1.score(TFVects_test,Y_test))

84% accuracy seems like the best we can get by using a logistic regression.

Now let's try with Naive Bayes.

In [ ]:
#wc = wordCounts(train_data, 'normalized_text')
wc = count_words(train_data, 'normalized_text', 'IsObama')
wc['veteran']

In [ ]:
classifier = NaiveBayesClassifier()
classifier.train(train_data, 'IsObama', 'normalized_text')

Let's evaluate the result on the test data.

In [ ]:
classified = []
for index, row in test_data.iterrows():
    classified.append((row['normalized_text'], row['IsObama'], classifier.classify(row['normalized_text'])))

# assume that spam_probability > 0.5 corresponds to spam prediction # and count the combinations of (actual is_spam, predicted is_spam)
counts = collections.Counter((actual, predicted_probability > 0.5)
                     for text, actual, predicted_probability in classified)

counts

Precision:

In [ ]:
precision = counts[(True,True)]/(counts[(False,True)]+counts[(True,True)]) #True positives over all positive predictions
print(precision)

Recall:

In [ ]:
recall = counts[(True,True)]/(counts[(True,False)]+counts[(True,True)])#what fraction of positives identified
print(recall)

F-measure:

In [ ]:
f_measure = 2 * (precision * recall)/(precision + recall)
print (f_measure)

In [ ]:
df_classification = pandas.DataFrame(classified, columns = ['press release', 'is Obama', 'posterior probability'])
df_classification = df_classification.round(2)

Let's take a look at how well our posterior distribution looks relative to the
truth.

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.grid(False)
ax1.set_frame_on(False)
df_classification[df_classification['is Obama'] == True]['posterior probability'].hist(alpha = 0.5, ax = ax1, bins = 10, label = 'Obama', color = 'red')
df_classification[df_classification['is Obama'] == False]['posterior probability'].hist(alpha = 0.5, ax = ax1, bins = 10, label = 'Clinton', color = 'blue')
ax1.set_xlim((0,1.1))
ax1.legend()
ax1.set_xlabel('posterior')
ax1.set_ylabel('counts')
plt.show()

The classification is suprisingly accurate.

In [ ]:
def p_obama_given_word(word_prob):
    """uses bayes's theorem to compute p(spam | message contains word)"""
    # word_prob is one of the triplets produced by word_probabilities

    word, prob_if_obama, prob_if_not_obama = word_prob
    return prob_if_obama / (prob_if_obama + prob_if_not_obama)

In [ ]:
words = sorted(classifier.word_probs,key=p_obama_given_word)

In [ ]:
Obama_words = words[-15:]
Clinton_words = words[:15]

What are the words that distinguish Obama and Clinton press releases?

In [ ]:
Obama_words

In [ ]:
Clinton_words

## Multinomial Naive Bayes

What if we want to classify our text into one of *many* classes? The multinomial
Naive Bayes generating model assumes that document features (e.g., words) are
generated by draws from a multinomial distribution (recall this gives the
probability to observe a particular pattern of counts across features).

Let's use again the dataset we used in week 3, the 20 newsgroup dataset.

In [ ]:
data = fetch_20newsgroups(data_home = 'data') #Free data to play with: documents from a newsgroup corpus.
data.target_names #Possible categories, i.e., the newsgroups

This dataset has a built in breakdown into training and testing sets. We can
pick specific categories, and pull the relevant training and testing sets.

In [ ]:
categories = ['talk.religion.misc', 'soc.religion.christian', 'sci.space', 'comp.graphics'] #Can change these of course
train = fetch_20newsgroups(subset='train', categories=categories, data_home = 'data')
test = fetch_20newsgroups(subset='test', categories=categories, data_home = 'data')

In [ ]:
train['target']

In [ ]:
len(train.data) #See how many training examples

In [ ]:
len(test.data) #Ditto for testing -- it's about 60% training, 40% testing

In [ ]:
print(train.data[5]) #Look at an example
#print(train.target_names[5])

We need to extract features from the text. We can use built-in feature
extraction to do so. We will use a tf-idf vectorizer, which converts the
document into a vector of words with tf-idf weights (term-frequency inverse-
document frequency). This gives high weight to words that show up a lot in a
given document but rarely across documents in the corpus (more distinctive).

We also take advantage of a useful feature of Scikit-Learn: we can make
pipelines...

In [ ]:
model = make_pipeline(TfidfVectorizer(max_df=100, min_df=2, stop_words='english', norm='l2'), MultinomialNB()) #This applies the vectorizer, then trains Multinomial NB

In [ ]:
model.fit(train.data,train.target) #Training syntax: feed the fit method the training data and the training targets

In [ ]:
labels = model.predict(test.data)

In [ ]:
labels.size

We can even use a confusion matrix, like we used last week for evaluating human
coders relative to one another. Now we are evaluating our classifier relative to
human coding.

In [ ]:
mat = confusion_matrix(test.target, labels)
seaborn.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
                xticklabels=train.target_names, yticklabels=train.target_names)
plt.xlabel('true label')
plt.ylabel('predicted label');

Let's calculate the precision, recall, and F-measures.

In [ ]:
print(sklearn.metrics.precision_score(test.target, labels, average = 'weighted')) #precision
print(sklearn.metrics.recall_score(test.target, labels, average = 'weighted')) #recall
print(sklearn.metrics.f1_score(test.target, labels, average = 'weighted')) #F-1 measure

Get the ROC curves.
First, we need to binarize the labels.

In [ ]:
lb = preprocessing.LabelBinarizer()
lb.fit(test.target)
print(lb.classes_)
print(lb.transform([1, 3]))

In [ ]:
y_test = lb.transform(test.target)
y_score = model.predict_proba(test.data)

In [ ]:
n_classes = 4

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = sklearn.metrics.roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = sklearn.metrics.auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = sklearn.metrics.roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = sklearn.metrics.auc(fpr["micro"], tpr["micro"])

In [ ]:
lw = 2
n_classes = 4
# Compute macro-average ROC curve and ROC area

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += sp.interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] =  sklearn.metrics.auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure()
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = ['aqua', 'darkorange', 'cornflowerblue', 'olive']
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()

We can also give the model a string and use the predict method to see if it can
assign it to a category. Using our model to extend to new, uncoded data might be
the primary purpose of a social science application.

In [ ]:
def predict_category(s, train=train, model=model): #We just define a simple function here
    return train.target_names[model.predict([s])]

In [ ]:
#predict_category('rockets')

Try it yourself with your own strings!


## Now let's add Bernie Sanders to our Obama / Clinton face-off

In [ ]:
ObamaClintonSandersReleases = pandas.read_csv("data/ObamaClintonSandersReleases.csv")
ObamaClintonSandersReleases = ObamaClintonSandersReleases.dropna(axis=0, how='any')

Let's split the data into training data and test data.

In [ ]:
data = list(zip(ObamaClintonSandersReleases['text'], ObamaClintonSandersReleases['targetSenator']))
random.seed(0) #This is important for replicability
train_data,test_data = split_data(data,0.75)

In [ ]:
print (len(train_data))
print (len(test_data))

In [ ]:
train.data = [data[0] for data in train_data]
train.target = [data[1] for data in train_data]
test.data = [data[0] for data in test_data]
test.target = [data[1] for data in test_data]

In [ ]:
model.fit(train.data,train.target) #Training syntax: feed the fit method the training data and the training targets

In [ ]:
labels = model.predict(test.data)

In [ ]:
labels.size

In [ ]:
model.classes_

Confusion matrix:

In [ ]:
mat = confusion_matrix(test.target, labels)
seaborn.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
                xticklabels=['Obama', 'Clinton', 'Sanders'], yticklabels=['Obama', 'Clinton', 'Sanders'])
plt.xlabel('true label')
plt.ylabel('predicted label');

Let's get the precision, recall, and F-measure.

In [ ]:
print(sklearn.metrics.precision_score(test.target, labels, average = 'weighted')) #precision
print(sklearn.metrics.recall_score(test.target, labels, average = 'weighted')) #recall
print(sklearn.metrics.f1_score(test.target, labels, average = 'weighted')) #F-1 measure

In order to get the ROC curves, first we need to binarize the labels.

In [ ]:
lb = preprocessing.LabelBinarizer()
lb.fit(test.target)
print(lb.classes_)
print(lb.transform(['Clinton', 'Obama']))

In [ ]:
y_test = lb.transform(test.target)
y_score = model.predict_proba(test.data)

In [ ]:
n_classes = 3

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = sklearn.metrics.roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = sklearn.metrics.auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = sklearn.metrics.roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = sklearn.metrics.auc(fpr["micro"], tpr["micro"])

In [ ]:
n_classes = 3
# Compute macro-average ROC curve and ROC area

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += sp.interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] =  sklearn.metrics.auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure()
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = ['aqua', 'darkorange', 'cornflowerblue']
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()

We can also give the model a string and use the predict method assign it to a
category. This extrapolation to new, untagged data will likely be the main point
of a social science application.

In [ ]:
model.predict(['money'])

In [ ]:
model.predict(['Wall Street'])

In [ ]:
model.predict(['Chicago'])

# Decision Trees

We now return to the python Machine learning package, Scikit-Learn, for a
decision tree example. Decision trees can be used to predict both
categorical/class labels (i.e., classification) and continuous labels (i.e.,
regression).

To illustrate how these work, we first create some artificial data to learn:

In [ ]:
X, y = make_blobs(n_samples=300, centers=4,
                random_state=0, cluster_std=1.0)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='rainbow');

Now we import our Decision Tree classifier from sklearn.tree (familiar syntax)
and fit it using the fit method.

In [ ]:
tree = DecisionTreeClassifier(max_depth=4,random_state=0).fit(X,y)

To see what's going on visually with the classification, we can use this
(complex) visualizer.

In [ ]:
def visualize_classifier(model, X, y, Xmod, ymod, ax=None, cmap='rainbow'): #X and y are plotted; Xmod and ymod train
    ax = ax or plt.gca()
    # Plot the training points
    ax.scatter(X[:, 0], X[:, 1], c=y, s=30, cmap=cmap,
               clim=(y.min(), y.max()), zorder=3)
    ax.axis('tight')
    ax.axis('off')
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    # fit the estimator
    model.fit(Xmod, ymod)
    xx, yy = np.meshgrid(np.linspace(*xlim, num=200),
                         np.linspace(*ylim, num=200))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
    # Create a color plot with the results
    n_classes = len(np.unique(y))
    contours = ax.contourf(xx, yy, Z, alpha=0.3,
                           levels=np.arange(n_classes + 1) - 0.5,
                           cmap=cmap, clim=(y.min(), y.max()),
                           zorder=1)
    ax.set(xlim=xlim, ylim=ylim)

In [ ]:
Xnew, ynew = make_blobs(n_samples=1000, centers=4,
                random_state=0, cluster_std=1.0)

In [ ]:
visualize_classifier(DecisionTreeClassifier(max_depth=4,random_state=0), Xnew, ynew, X, y) #We train on the full data

In [ ]:
accuracy_score(ynew,DecisionTreeClassifier(max_depth=10,random_state=0).fit(X,y).predict(Xnew))

In [ ]:
depthvec = []
scorevec = []
for i in range(1,20):
    tree2 = DecisionTreeClassifier(max_depth=i,random_state=0).fit(X,y)
    score = accuracy_score(ynew,tree2.predict(Xnew))
    depthvec.append(i)
    scorevec.append(score)
plt.scatter(depthvec,scorevec)

We an select different layers of the tree or "prune" the decision tree. At
approximately four layers down the decision tree, the shape is somewhat odd,
suggesting that our model is overfitting beyond those four layers.

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.5,random_state=1) #test_size means a 50/50 split
print(len(X))
print(len(Xtrain))
print(len(Xtest))

With some abuse of notation, we can use the visualizer to plot first a
classifier trained on the "training" half of the data.

In [ ]:
visualize_classifier(DecisionTreeClassifier(max_depth=5,random_state=0), X, y, Xtrain, ytrain) #Train with half the data

Now we can train on the second half of the data--allegedly the test data--to see
how different training sets affect the decision boundaries.

In [ ]:
visualize_classifier(DecisionTreeClassifier(random_state=0), X, y, Xtest, ytest) #Train with other half of the data

Finally, we can get a sense of the performance by training on the training data,
but PLOTTING the test data

In [ ]:
visualize_classifier(DecisionTreeClassifier(), Xtest, ytest, Xtrain, ytrain) #Trains with train data, plots test data

Combining multiple overfitting estimators turns out to be a key idea in machine
learning. This is called **bagging** and is a type of **ensemble** method. The
idea is to make many randomized estimators--each can overfit, as decision trees
are wont to do--but then to combine them, ultimately producing a better
classification. A **random forest** is produced by bagging decision trees.

In [ ]:
tree = DecisionTreeClassifier(max_depth=10) #Create an instance of our decision tree classifier.

bag = BaggingClassifier(tree, n_estimators=100, max_samples=0.8, random_state=1) #Each tree uses up to 80% of the data

In [ ]:
#?BaggingClassifier #Learn more

In [ ]:
bag.fit(X,y) #Fit the bagged classifier

In [ ]:
Xnew, ynew = make_blobs(n_samples=10000, centers=4,
                random_state=0, cluster_std=1.0)

In [ ]:
visualize_classifier(bag,Xnew,ynew,X,y) #And visualize
#Remember we can give the full data as training data, as bag automatically splits and trains

In [ ]:
recall_score(ynew,bag.predict(Xnew),average='weighted')

## Let's try this on the Clinton / Obama / Sanders dataset.

In [ ]:
TFVects = TFVectorizer.fit_transform(train.data)

In [ ]:
tree = DecisionTreeClassifier(max_depth=4,random_state=0).fit(TFVects,train.target)

In [ ]:
TFVects_test = TFVectorizer.transform(test.data)

In [ ]:
labels = tree.predict(TFVects_test)

Confusion matrix:

In [ ]:
mat = confusion_matrix(test.target, labels)
seaborn.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
                xticklabels=['Obama', 'Clinton', 'Sanders'], yticklabels=['Obama', 'Clinton', 'Sanders'])
plt.xlabel('true label')
plt.ylabel('predicted label');

The precision, recall, and F-measure.

In [ ]:
print(sklearn.metrics.precision_score(test.target, labels, average = 'weighted')) #precision
print(sklearn.metrics.recall_score(test.target, labels, average = 'weighted')) #recall
print(sklearn.metrics.f1_score(test.target, labels, average = 'weighted')) #F-1 measure

The decision tree is not substantially better than Multinomial Naive Bayes.
Let's try a random forest.

In [ ]:
bag.fit(TFVects,train.target) #Fit the bagged classifier

In [ ]:
labels = bag.predict(TFVects_test)

In [ ]:
mat = confusion_matrix(test.target, labels)
seaborn.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
                xticklabels=['Obama', 'Clinton', 'Sanders'], yticklabels=['Obama', 'Clinton', 'Sanders'])
plt.xlabel('true label')
plt.ylabel('predicted label');

In [ ]:
print(sklearn.metrics.precision_score(test.target, labels, average = 'weighted')) #precision
print(sklearn.metrics.recall_score(test.target, labels, average = 'weighted')) #recall
print(sklearn.metrics.f1_score(test.target, labels, average = 'weighted')) #F-1 measure

The performance is better!


# K-Nearest Neighbors

The K-Nearest neighbors classifier takes a simpler premise than those before:
Find the labeled datapoint in question and "borrow" its label.

Let's use newsgroup data again.

In [ ]:
categories = ['soc.religion.christian', 'sci.space', 'comp.graphics'] #Can change these of course
train = fetch_20newsgroups(subset='train', categories=categories, data_home = 'data')
test = fetch_20newsgroups(subset='test', categories=categories, data_home = 'data')

Let's visualize it using PCA.

In [ ]:
TFVectorizer = sklearn.feature_extraction.text.TfidfVectorizer(max_df=0.5, min_df=3, stop_words='english', norm='l2')
TFVects = TFVectorizer.fit_transform(train.data)

In [ ]:
TFVects.shape

In [ ]:
reduced_data = PCA(n_components = 2).fit_transform(TFVects.toarray())

In [ ]:
train.target_names

Visualization:

In [ ]:
colordict = {
0: 'red',
1: 'orange',
2: 'green',
    }
colors = [colordict[c] for c in train.target]
fig = plt.figure(figsize = (10,6))
ax = fig.add_subplot(111)
ax.set_frame_on(False)
ax.scatter(reduced_data[:, 0], reduced_data[:, 1], color = colors, alpha = 0.5, label = colors)
plt.xticks(())
plt.yticks(())
plt.title('True Classes, Training Set')
plt.show()

Let's initialize our k-nearest neighbors classifier.

In [ ]:
n_neighbors = 15
weights="uniform"
clf = neighbors.KNeighborsClassifier(n_neighbors, weights=weights)

For visualization purpopose, let's first try applying the k-nearest neighbors
classifier to the projected data.

In [ ]:
clf.fit(reduced_data, train.target)

In [ ]:
TFVects_test = TFVectorizer.transform(test.data)
reduced_data_test = PCA(n_components = 2).fit_transform(TFVects_test.toarray())

In [ ]:
labels = clf.predict(reduced_data_test)

Confusion matrix:

In [ ]:
mat = confusion_matrix(test.target, labels)
seaborn.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
                xticklabels=categories, yticklabels=categories)
plt.xlabel('true label')
plt.ylabel('predicted label');

The precision, recall, and F-measure.

In [ ]:
print(sklearn.metrics.precision_score(test.target, labels, average = 'weighted')) #precision
print(sklearn.metrics.recall_score(test.target, labels, average = 'weighted')) #recall
print(sklearn.metrics.f1_score(test.target, labels, average = 'weighted')) #F-1 measure

In [ ]:
h = .02  # step size in the mesh
# Create color maps
cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])
cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])

x_min, x_max = reduced_data[:, 0].min() - 0.2, reduced_data[:, 0].max() + 0.2
y_min, y_max = reduced_data[:, 1].min() - 0.2, reduced_data[:, 1].max() + 0.2
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure()
plt.pcolormesh(xx, yy, Z, cmap=cmap_light)

# Plot also the training points
plt.scatter(reduced_data[:, 0], reduced_data[:, 1], c=colors, cmap=cmap_bold)
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.title("3-Class classification (k = %i, weights = '%s')"
          % (n_neighbors, weights))

plt.show()

Let's use the full tf-idf matrix instead.

In [ ]:
clf.fit(TFVects, train.target)

In [ ]:
labels = clf.predict(TFVects_test)

Confusion matrix:

In [ ]:
mat = confusion_matrix(test.target, labels)
seaborn.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
                xticklabels=categories, yticklabels=categories)
plt.xlabel('true label')
plt.ylabel('predicted label');

Small improvement. In this case, PCA reduced the dimensionality of the data
without loosing too much informatin.
Finally, let's use it again on the Obama Clinton Sanders dataset.

In [ ]:
train.data = [data[0] for data in train_data]
train.target = [data[1] for data in train_data]
test.data = [data[0] for data in test_data]
test.target = [data[1] for data in test_data]

In [ ]:
TFVects_OCS = TFVectorizer.fit_transform(train.data)
TFVects_OCS_test = TFVectorizer.transform(test.data)

In [ ]:
clf.fit(TFVects_OCS,train.target)

In [ ]:
labels = clf.predict(TFVects_OCS_test)

Confusion matrix:

In [ ]:
mat = confusion_matrix(test.target, labels)
seaborn.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
                xticklabels=['Obama', 'Clinton', 'Sanders'], yticklabels=['Obama', 'Clinton', 'Sanders'])
plt.xlabel('true label')
plt.ylabel('predicted label');

In [ ]:
print(sklearn.metrics.precision_score(test.target, labels, average = 'weighted')) #precision
print(sklearn.metrics.recall_score(test.target, labels, average = 'weighted')) #recall
print(sklearn.metrics.f1_score(test.target, labels, average = 'weighted')) #F-1 measure

# SVMs

Now we will examine Support Vector Machines, an approach that creates the
partition that preserves the maximum margin between classes.

We will use a few sub forums from reddit--which tend to share text rather than
memes--namely `talesfromtechsupport`, `badroommates`, `weeabootales` and
`relationships`. The top 100 text posts from each have been saved to
`data/reddit.csv`.

In [ ]:
redditDf = pandas.read_csv('data/reddit.csv')

#Drop a couple missing values

redditDf = redditDf.dropna()

#tokenize and normalize
redditDf['tokenized_text'] = redditDf['text'].apply(lambda x: nltk.word_tokenize(x))
redditDf['normalized_text'] = redditDf['tokenized_text'].apply(lambda x: normlizeTokens(x, stopwordLst = stop_words_nltk, stemmer = snowball))

We will tf.idf the data to make our vectors

In [ ]:
redditTFVectorizer = sklearn.feature_extraction.text.TfidfVectorizer(max_df=0.5, min_df=3, stop_words='english', norm='l2')
redditTFVects = redditTFVectorizer.fit_transform([' '.join(l) for l in redditDf['normalized_text']])
redditDf['tfVect'] = [np.array(v) for v in redditTFVects.todense()]
redditDf['tfVect'][:5]

In [ ]:
redditDf['tfVect'][0].shape

Initilize the model

In [ ]:
clf = sklearn.svm.SVC(kernel='linear', probability = False)
#probability = True is slower but  lets you call predict_proba()
clf

Setup a training set

In [ ]:
np.random.seed(0)
shuffled_redditDf = redditDf.reindex(np.random.permutation(redditDf.index))
holdBackFraction = .2
holdBackIndex = int(holdBackFraction * len(shuffled_redditDf))
training_redditDf = shuffled_redditDf[holdBackIndex: ].copy()
test_redditDf = shuffled_redditDf[:holdBackIndex].copy()
print(len(training_redditDf))
print(len(test_redditDf))

Fit the model

In [ ]:
clf.fit(np.stack(training_redditDf['tfVect'], axis=1)[0], training_redditDf['subreddit'].as_matrix())

Here are the 4 classes of posts:

In [ ]:
clf.classes_

And a prediction from the testing set

In [ ]:
#print(clf.predict_proba(test_redditDf['tfVect'][82]))
print(clf.predict(test_redditDf['tfVect'][82]))
print(test_redditDf['subreddit'][82])

In [ ]:
redditDf['svc'] = [clf.predict(v)[0] for v in redditDf['tfVect']]

In [ ]:
mat = confusion_matrix(redditDf['svc'], redditDf['subreddit'])
seaborn.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
                xticklabels=redditDf['subreddit'].unique(), yticklabels=redditDf['subreddit'].unique())
plt.xlabel('true label')
plt.ylabel('predicted label')

# Neural Nets

We include an example of a simple neural network, the Multi-layer Perceptron
(MLP) that learns a function $f(\cdot): R^m \rightarrow R^o$ by training on a
dataset, where $m$ is the number of dimensions for input and $o$ is the number
of dimensions for output. Given a set of features $X = {x_1, x_2, ..., x_m}$ and
a target $y$, it can learn a non-linear function approximator for either
classification or regression. It is different from logistic regression, in that
between the input and the output layer, there can be one or more non-linear
layers, called hidden layers. The following figure shows a one hidden layer MLP
with scalar output. ![title](data/multilayerperceptron_network.png) The leftmost
layer, known as the input layer, consists of a set of "neurons" $\{x_i | x_1,
x_2, ..., x_m\}$ representing the input features (e.g., weighted words). Each
neuron in the hidden layer transforms the values from the previous layer with a
weighted linear summation $w_1x_1 + w_2x_2 + ... + w_mx_m$, followed by a non-
linear activation function $g(\cdot):R \rightarrow R$ - like the logistic or
hyperbolic tan function. The output layer receives the values from the last
hidden layer and transforms them into output values.

In [ ]:
clf = MLPClassifier()
clf

In [ ]:
clf.fit(np.stack(training_redditDf['tfVect'], axis=1)[0], training_redditDf['subreddit'].as_matrix())

In [ ]:
clf.classes_

In [ ]:
print(clf.predict_proba(test_redditDf['tfVect'][82]))
print(clf.predict(test_redditDf['tfVect'][82]))
print(test_redditDf['subreddit'][82])

In [ ]:
test_redditDf['MLP'] = [clf.predict(v)[0] for v in test_redditDf['tfVect']]

In [ ]:
mat = confusion_matrix(test_redditDf['MLP'], test_redditDf['subreddit'])
seaborn.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
                xticklabels=redditDf['subreddit'].unique(), yticklabels=redditDf['subreddit'].unique())
plt.xlabel('true label')
plt.ylabel('predicted label')

# NLP

We will be using [`nltk`](http://www.nltk.org/) for this notebook this is fine for an introduction but the state of the are implementations are done by the [Stanford NLP group](http://nlp.stanford.edu/). These can even be integrated into Python but are much slower and computationally intensive, for more on integrating look [here](https://github.com/KnowledgeLab/content_analysis/blob/release/7-Information-Extraction/7-Information-Extraction.ipynb).



First, we will illustrate these tools on some *very* short examples:

In [4]:
text = ['I saw the elephant in my pajamas.', 'The quick brown fox jumped over the lazy dog.', 'While in France, Christine Lagarde discussed short-term stimulus efforts in a recent interview with the Wall Street Journal.', 'Trayvon Benjamin Martin was an African American from Miami Gardens, Florida, who, at 17 years old, was fatally shot by George Zimmerman, a neighborhood watch volunteer, in Sanford, Florida.', 'Buffalo buffalo Buffalo buffalo buffalo buffalo Buffalo buffalo']
tokenized_text = [word_tokenize(t) for t in text]
print('\n'.join(text))

I saw the elephant in my pajamas.
The quick brown fox jumped over the lazy dog.
While in France, Christine Lagarde discussed short-term stimulus efforts in a recent interview with the Wall Street Journal.
Trayvon Benjamin Martin was an African American from Miami Gardens, Florida, who, at 17 years old, was fatally shot by George Zimmerman, a neighborhood watch volunteer, in Sanford, Florida.
Buffalo buffalo Buffalo buffalo buffalo buffalo Buffalo buffalo


# Part-of-Speech (POS) tagging

In POS tagging, we classify each word by its semantic role in a sentence. The
Stanford POS tagger uses the [Penn Treebank tag set]('http://repository.upenn.ed
u/cgi/viewcontent.cgi?article=1603&context=cis_reports') to POS tag words from
input sentences. As discussed in the second assignment, this is a relatively
precise tagset, which allows more informative tags, and also more opportunities
to err :-).

|#. |Tag |Description |
|---|----|------------|
|1.     |CC     |Coordinating conjunction
|2.     |CD     |Cardinal number
|3.     |DT     |Determiner
|4.     |EX     |Existential there
|5.     |FW     |Foreign word
|6.     |IN     |Preposition or subordinating conjunction
|7.     |JJ     |Adjective
|8.     |JJR|   Adjective, comparative
|9.     |JJS|   Adjective, superlative
|10.|   LS      |List item marker
|11.|   MD      |Modal
|12.|   NN      |Noun, singular or mass
|13.|   NNS     |Noun, plural
|14.|   NNP     |Proper noun, singular
|15.|   NNPS|   Proper noun, plural
|16.|   PDT     |Predeterminer
|17.|   POS     |Possessive ending
|18.|   PRP     |Personal pronoun
|19.|   PRP\$|  Possessive pronoun
|20.|   RB      |Adverb
|21.|   RBR     |Adverb, comparative
|22.|   RBS     |Adverb, superlative
|23.|   RP      |Particle
|24.|   SYM     |Symbol
|25.|   TO      |to
|26.|   UH      |Interjection
|27.|   VB      |Verb, base form
|28.|   VBD     |Verb, past tense
|29.|   VBG     |Verb, gerund or present participle
|30.|   VBN     |Verb, past participle
|31.|   VBP     |Verb, non-3rd person singular present
|32.|   VBZ     |Verb, 3rd person singular present
|33.|   WDT     |Wh-determiner
|34.|   WP      |Wh-pronoun
|35.|   WP$     |Possessive wh-pronoun
|36.|   WRB     |Wh-adverb

In [9]:
nltk.pos_tag('a bird')

pos_sents = [nltk.pos_tag(s) for s in tokenized_text]
for s in pos_sents:
    print(s)

[('I', 'PRP'), ('saw', 'VBD'), ('the', 'DT'), ('elephant', 'NN'), ('in', 'IN'), ('my', 'PRP$'), ('pajamas', 'NN'), ('.', '.')]
[('The', 'DT'), ('quick', 'JJ'), ('brown', 'NN'), ('fox', 'NN'), ('jumped', 'VBD'), ('over', 'IN'), ('the', 'DT'), ('lazy', 'JJ'), ('dog', 'NN'), ('.', '.')]
[('While', 'IN'), ('in', 'IN'), ('France', 'NNP'), (',', ','), ('Christine', 'NNP'), ('Lagarde', 'NNP'), ('discussed', 'VBD'), ('short-term', 'JJ'), ('stimulus', 'NN'), ('efforts', 'NNS'), ('in', 'IN'), ('a', 'DT'), ('recent', 'JJ'), ('interview', 'NN'), ('with', 'IN'), ('the', 'DT'), ('Wall', 'NNP'), ('Street', 'NNP'), ('Journal', 'NNP'), ('.', '.')]
[('Trayvon', 'NNP'), ('Benjamin', 'NNP'), ('Martin', 'NNP'), ('was', 'VBD'), ('an', 'DT'), ('African', 'JJ'), ('American', 'JJ'), ('from', 'IN'), ('Miami', 'NNP'), ('Gardens', 'NNP'), (',', ','), ('Florida', 'NNP'), (',', ','), ('who', 'WP'), (',', ','), ('at', 'IN'), ('17', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('was', 'VBD'), ('fatally', 'RB')

This looks quite good. Now we will try POS tagging with a somewhat larger
corpus. We consider a few of the top posts from the reddit data we used last
week.

In [10]:
redditDF = pandas.read_csv('data/reddit.csv')

Grabbing the 10 highest scoring posts and tokenizing the sentences. Once again,
notice that we aren't going to do any kind of stemming this week (although
*semantic* normalization may be performed where we translate synonyms into the
same focal word).

In [11]:
redditTopScores = redditDF.sort_values('score')[-10:]
redditTopScores['sentences'] = redditTopScores['text'].apply(lambda x: [nltk.word_tokenize(s) for s in nltk.sent_tokenize(x)])
redditTopScores.index = range(len(redditTopScores) - 1, -1,-1) #Reindex to make things nice in the future
redditTopScores

,Unnamed: 0,author,over_18,score,subreddit,text,title,url,sentences
9,8,brenkelieshere,False,9448,Tales From Tech Support,"Last year, Help Desk got a call from a user co...",How to fix a laptop that won't boot in under a...,https://www.reddit.com/r/talesfromtechsupport/...,"[[Last, year, ,, Help, Desk, got, a, call, fro..."
8,7,Bombadils,False,10528,Tales From Tech Support,First post in quite some time! I work at a loc...,"OK, now the password is 'D35p41r'",https://www.reddit.com/r/talesfromtechsupport/...,"[[First, post, in, quite, some, time, !], [I, ..."
7,1200,whenlifegivesyoushit,False,11003,Relationships,[Original Post](https://www.reddit.com/r/relat...,[UPDATE]My [26 F] with my husband [29 M] 1 yea...,https://www.reddit.com/r/relationships/comment...,"[[[, Original, Post, ], (, https, :, //www.red..."
6,6,sfsdfd,False,11295,Tales From Tech Support,I witnessed this astounding IT meltdown around...,"Company-wide email + 30,000 employees + auto-r...",https://www.reddit.com/r/talesfromtechsupport/...,"[[I, witnessed, this, astounding, IT, meltdown..."
5,5,Raitaro,False,12372,Tales From Tech Support,I work Helpdesk for a retail store chain in th...,I'm pretty sure I knocked a user out from near...,https://www.reddit.com/r/talesfromtechsupport/...,"[[I, work, Helpdesk, for, a, retail, store, ch..."
4,4,goldie-gold,False,12650,Tales From Tech Support,"This just happened... So, I had a laptop syst...",Engineer is doing drugs!! No. No they aren't.,https://www.reddit.com/r/talesfromtechsupport/...,"[[This, just, happened, ...], [So, ,, I, had, ..."
3,3,TheDroolinFool,False,13152,Tales From Tech Support,Another tale from the out of hours IT desk... ...,"""I need you to fix Google Bing immediately!""",https://www.reddit.com/r/talesfromtechsupport/...,"[[Another, tale, from, the, out, of, hours, IT..."
2,2,Clickity_clickity,False,13404,Tales From Tech Support,[Part 1](http://www.reddit.com/r/talesfromtech...,"Jack, the Worst End User, Part 4",https://www.reddit.com/r/talesfromtechsupport/...,"[[[, Part, 1, ], (, http, :, //www.reddit.com/..."
1,1,SECGaz,False,13724,Tales From Tech Support,"> $Me - Hello, IT. > $Usr - Hi, I am still ...","Hi, I am still off sick but I am not.",https://www.reddit.com/r/talesfromtechsupport/...,"[[>, $, Me, -, Hello, ,, IT, .], [>, $, Usr, -..."
0,0,guitarsdontdance,False,14089,Tales From Tech Support,So my story starts on what was a normal day ta...,"""Don't bother sending a tech, I'll be dead by ...",https://www.reddit.com/r/talesfromtechsupport/...,"[[So, my, story, starts, on, what, was, a, nor..."


In [14]:
redditTopScores['POS_sents'] = redditTopScores['sentences'].apply(lambda x: [nltk.pos_tag(s) for s in x])

In [16]:
redditTopScores['POS_sents']

9    [[(Last, JJ), (year, NN), (,, ,), (Help, NNP),...
8    [[(First, RB), (post, NN), (in, IN), (quite, R...
7    [[([, JJ), (Original, NNP), (Post, NNP), (], N...
6    [[(I, PRP), (witnessed, VBD), (this, DT), (ast...
5    [[(I, PRP), (work, VBP), (Helpdesk, NNP), (for...
4    [[(This, DT), (just, RB), (happened, VBN), (.....
3    [[(Another, DT), (tale, NN), (from, IN), (the,...
2    [[([, JJ), (Part, NNP), (1, CD), (], NNP), ((,...
1    [[(>, RB), ($, $), (Me, NNP), (-, :), (Hello, ...
0    [[(So, RB), (my, PRP$), (story, NN), (starts, ...
Name: POS_sents, dtype: object

And count the number of `NN` (nouns)

In [17]:
countTarget = 'NN'
targetCounts = {}
for entry in redditTopScores['POS_sents']:
    for sentence in entry:
        for ent, kind in sentence:
            if kind != countTarget:
                continue
            elif ent in targetCounts:
                targetCounts[ent] += 1
            else:
                targetCounts[ent] = 1
sortedTargets = sorted(targetCounts.items(), key = lambda x: x[1], reverse = True)
sortedTargets[:20]

[('password', 21),
 ('time', 14),
 ('lot', 12),
 ('computer', 12),
 ('life', 11),
 ('**Genius**', 10),
 ('email', 10),
 ('message', 9),
 ('system', 9),
 ('**Me**', 9),
 ('day', 9),
 ('today', 8),
 ('office', 8),
 ('part', 8),
 ('call', 8),
 ('story', 8),
 ('file', 7),
 ('work', 7),
 ('user', 7),
 ('problem', 7)]

What about the number of top verbs (`VB`)?

In [18]:
countTarget = 'VB'
targetCounts = {}
for entry in redditTopScores['POS_sents']:
    for sentence in entry:
        for ent, kind in sentence:
            if kind != countTarget:
                continue
            elif ent in targetCounts:
                targetCounts[ent] += 1
            else:
                targetCounts[ent] = 1
sortedTargets = sorted(targetCounts.items(), key = lambda x: x[1], reverse = True)
sortedTargets[:20]

[('be', 18),
 ('have', 17),
 ('get', 14),
 ('do', 11),
 ('make', 8),
 ('change', 8),
 ('say', 7),
 ('know', 6),
 ('send', 6),
 ('look', 6),
 ('tell', 6),
 ('help', 5),
 ('receive', 4),
 ('thank', 4),
 ('go', 4),
 ('remember', 4),
 ('call', 4),
 ('open', 4),
 ('take', 4),
 ('want', 4)]

What about the adjectives that modify the word, "computer"?

In [19]:
NTarget = 'JJ'
Word = 'computer'
NResults = set()
for entry in redditTopScores['POS_sents']:
    for sentence in entry:
        for (ent1, kind1),(ent2,kind2) in zip(sentence[:-1], sentence[1:]):
            if (kind1,ent2.lower())==(NTarget,Word):
                NResults.add(ent1)
            else:
                continue

print(NResults)     

{'own', 'C-O-M-P-U-T-E-R.', 'unrestricted'}


## Evaluating POS tagger

We can check the POS tagger by running it on a manually tagged corpus and
identifying a reasonable error metric.

In [20]:
treeBank = nltk.corpus.treebank
treeBank.tagged_sents()[0]

[('Pierre', 'NNP'),
 ('Vinken', 'NNP'),
 (',', ','),
 ('61', 'CD'),
 ('years', 'NNS'),
 ('old', 'JJ'),
 (',', ','),
 ('will', 'MD'),
 ('join', 'VB'),
 ('the', 'DT'),
 ('board', 'NN'),
 ('as', 'IN'),
 ('a', 'DT'),
 ('nonexecutive', 'JJ'),
 ('director', 'NN'),
 ('Nov.', 'NNP'),
 ('29', 'CD'),
 ('.', '.')]

In [21]:
treeBank.sents()[0]

['Pierre',
 'Vinken',
 ',',
 '61',
 'years',
 'old',
 ',',
 'will',
 'join',
 'the',
 'board',
 'as',
 'a',
 'nonexecutive',
 'director',
 'Nov.',
 '29',
 '.']

In [24]:
nltkTags = [nltk.pos_tag(s) for s in treeBank.sents()[:30]]

And compare the two

In [25]:
NumDiffs = 0
for sentIndex in range(len(nltkTags)):
    for wordIndex in range(len(nltkTags[sentIndex])):
        if nltkTags[sentIndex][wordIndex][1] != treeBank.tagged_sents()[sentIndex][wordIndex][1]:
            if treeBank.tagged_sents()[sentIndex][wordIndex][1] != '-NONE-':
                print("Word: {}  \tStanford: {}\tTreebank: {}".format(nltkTags[sentIndex][wordIndex][0], nltkTags[sentIndex][wordIndex][1], treeBank.tagged_sents()[sentIndex][wordIndex][1]))
                NumDiffs += 1
total = sum([len(s) for s in nltkTags])
print("The Precision is {:.3f}%".format((total-NumDiffs)/total * 100))

Word: publishing  	Stanford: NN	Treebank: VBG
Word: used  	Stanford: VBD	Treebank: VBN
Word: more  	Stanford: JJR	Treebank: RBR
Word: ago  	Stanford: RB	Treebank: IN
Word: that  	Stanford: IN	Treebank: WDT
Word: later  	Stanford: RB	Treebank: JJ
Word: New  	Stanford: NNP	Treebank: JJ
Word: that  	Stanford: IN	Treebank: WDT
Word: Micronite  	Stanford: NNP	Treebank: NN
Word: ago  	Stanford: RB	Treebank: IN
Word: ago  	Stanford: RB	Treebank: IN
Word: heard  	Stanford: NN	Treebank: VBD
Word: asbestos  	Stanford: NNS	Treebank: NN
Word: replaced  	Stanford: VBD	Treebank: VBN
Word: worked  	Stanford: VBN	Treebank: VBD
Word: more  	Stanford: RBR	Treebank: JJ
Word: expected  	Stanford: JJ	Treebank: VBN
Word: study  	Stanford: NN	Treebank: VBP
Word: asbestos  	Stanford: JJ	Treebank: NN
Word: studied  	Stanford: VBD	Treebank: VBN
Word: Western  	Stanford: NNP	Treebank: JJ
Word: industrialized  	Stanford: JJ	Treebank: VBN
Word: Lorillard  	Stanford: NNP	Treebank: NN
Word: argue  	Stanford: NN	Tree

So we can see that the stanford POS tagger is quite good. Nevertheless, for a 20
word sentence, we only have a 66% chance ($1-.96^{20}$) of tagging (and later
parsing) it correctly.

In [31]:
trees = nltk.chunk.ne_chunk_sents(redditTopScores['POS_sents'])

generator